In [111]:
import pandas as pd
import numpy as np
import re

from io import StringIO

from sklearn.externals import joblib
from scipy.sparse import csr_matrix, lil_matrix
from sklearn.datasets import make_regression
from sklearn.linear_model import SGDRegressor
from sklearn.linear_model import LinearRegression
from sklearn.feature_extraction import DictVectorizer


In [112]:
###читаем даатасет c фичами фильмов
df_f = pd.read_csv('./movie_titles.csv', names=['MovieID', 'Year', 'Title'])
df_f = df_f.set_index('MovieID')

###читаем даатасет с инфой о пользователях
f = open('./combined_data_1.txt')
file = f.read()

movieID_u = list(map(lambda x: int(x[:-1]), re.findall(r'\d+\:', file)))
df_u = re.split(r'\d+\:', file)
df_u = df_u[1:]    

df = []
for df_i, movieID_i in zip(df_u, movieID_u):
    sub_df = pd.read_csv(StringIO(df_i), names=['UserID', 'Score', 'Date'])
    sub_df['MovieID'] = movieID_i
    df.append(sub_df)


df = pd.concat(df)
df_f = df_f.drop(df_f[df_f.index>df.MovieID.unique().shape[0]].index)
#print(df_f)
#print(df)


In [113]:
u_size = df.UserID.unique().shape[0]
m_size = df.MovieID.unique().shape[0]
#y_size = df_f.Year.unique().shape[0]
#s_size = df.Date.unique().shape[0]

In [114]:
df.sort_values(by=['UserID', 'MovieID'], inplace=True)
df.reset_index(drop=True, inplace=True)
df_targ = df.Score

In [115]:
### разреженная матрица
sm = lil_matrix((df.shape[0], u_size + m_size))
#print(sm.shape)

In [116]:
j=0
temp = df.UserID.iat[0]

for i in range(df.shape[0]):
    
    if(temp<df.UserID.iat[i]):
        j+=1
        temp = df.UserID.iat[i]
        
    sm[i, j] = 1
    sm[i, u_size + df.MovieID.iat[i]-1] = 1
    
    if i % 50000 == 0:
        #print("\r Progress: {}/{} ({}%)".format(i, df.shape[0], int(((i+1) / df.shape[0]) * 100)), end="")
    #print(i, j, sm[i,j])
    #print(i, j1, sm[i,j1])
print(sm[0,0])

SyntaxError: unexpected EOF while parsing (<ipython-input-116-e9e3657227f4>, line 17)

In [117]:
sm = csr_matrix(sm)
joblib.dump(sm, 'sparse_df_1.bin')

['sparse_df_1.bin']

In [118]:
sm = joblib.load('sparse_df_1.bin')

In [119]:
print(sm.shape)
print(df_targ.shape)

(24053764, 475257)
(24053764,)


По лемме 1 нелинейную часть можно представить в виде: $\sum_{i=1}^n \sum_{j=i+1}^n \langle {v}_i, {v}_f\rangle x_ix_j = \frac{1}{2}\sum_{f=1}^k \left( \left(\sum_{i=1}^n v_{i,f}x_i \right)^2 - \sum_{i=1}^n v^2_{i,f}x^2_i \right)$
<br>
Производная $\frac{dy}{dv}$ нелинейной части бдует следующей:
<br>
(1) = $\left( \left( \sum_{i=1}^n v_{i,f}x_i \right)^2 \right)' _{v_if} = 2\sum_{i=1}^n \left(v_{i,f}x_i \sum_{k=1}^n x_k\right)$ или в матричном виде $\sum_{f=1}^k2 (X\mathbb{1})^TXV_f$
<br>
(2) = $ \left( \sum_{i=1}^n v_{i,f}^2x_i^2 \right)' _{v_if} = 2\sum_{i=1}^n v_{i,f}x_i^2 $ или в матричном виде $\sum_{f=1}^k2 X^TXV_f$
<br>
<br>
Тогда
<br>
$\frac{dy}{dv_f} = \frac{1}{2}\sum_{f=1}^k \left( (1) - (2) \right)$ или в матричном виде $\sum_{f=1}^k(X\mathbb{1}-X)^TXV_f$
<br>
<br>
И тогда $\frac{dy}{dv}$ в матричном виде $(X\mathbb{1}-X)^TXV\mathbb{1}$

In [120]:
def RMSE(x, y):
    return np.sqrt(np.sum(np.power(y - x, 2)) / y.shape[0])


def narrTheta(lr, iter_num):
    return lr / np.sqrt(iter_num)


def sgdFFM(X, y, max_iter=1e3, step=1e-3, epsilon=1e-6, batch_size=256, k=9):
    w0 = 0
    iter_num = 1
    N = X.shape[0]
    np.random.seed(42)
    w1 = np.random.normal(size=X.shape[1])
    V = np.random.normal(size=(X.shape[1], k))
    #V = np.zeros((X.shape[1], k))

    while iter_num <= max_iter:
        random_batch = np.random.choice(N, batch_size)
        new_y = predict(X[random_batch, :], V, w0, w1)
        # print(X[random_batch].shape)

        dy = 2 * (new_y - y[random_batch])
        # print(iter_num, dy)
        w0 -= dy.mean() * step
        w1 -= X[random_batch, :].T @ dy * step / batch_size#narrTheta(step, iter_num)
        for f in range(k):    
            # (256*8 . 256*1) = 256*8
            dA = X[random_batch, :]*(X[random_batch, :]@V[:,f]).reshape(-1, 1)
            #print(dA.shape)            
            #256*8x8*1 = 256*8
            dB = (X[random_batch, :]**2)*V[:,f]
            #print(dB.shape)
            #print(dy.shape)
            V[:,f] -= step*(dy.reshape(-1, 1)*(dA-dB)).mean(axis=0)
        
        iter_num += 1
        if (np.linalg.norm(y[random_batch] - new_y) < epsilon): 
            print(np.linalg.norm(y[random_batch] - new_y))
            return w0, w1, V
        
        if (iter_num%100==0):
            print(np.linalg.norm(y[random_batch] - new_y))
        
    return w0, w1, V

def nonLin(X, V):
    A = (X@V)**2
    B = (X**2)@(V**2)
    return 1/2*((A-B)@np.ones(V.shape[1]))

def predict(X, V, w0, w1):
    return w0 + X @ w1 + nonLin(X, V)


<h1>Проверим на генерации</h1>

In [121]:
X, y, coefs = make_regression(n_samples=10000, n_features=8, n_targets=1, n_informative=4, coef=True)

folds_index = 5
fold_size = round(X.shape[0] / folds_index)
RMSE_test = []
RMSE_train = []

for i in range(folds_index):
    test = X[i * fold_size:(i + 1) * fold_size]
    testT = y[i * fold_size:(i + 1) * fold_size]
    if i == 0:
        train = X[(i + 1) * fold_size:, :]
        trainT = y[(i + 1) * fold_size:]
    else:
        train = X[:i * fold_size, :]
        trainT = y[:i * fold_size]
        if i != 4:
            train = np.concatenate((train, X[(i + 1) * fold_size:, :]))
            trainT = np.concatenate([trainT, y[(i + 1) * fold_size:]])
    
    Target = trainT
    Features = train
    w0, w1, V = sgdFFM(Features, Target, 1e4, 1e-3)

    train_pred = predict(Features, V, w0, w1)

    RMSE_train.append(RMSE(train_pred, Target))
    TargetT = testT
    FeaturesT = test
    test_pred = predict(FeaturesT, V, w0, w1)
    RMSE_test.append(RMSE(test_pred, TargetT))

print('Fold {}'.format(i + 1), '\nОбучающая: \n', train_pred - Target, '\nТестовая: \n', test_pred - TargetT, '\n', '\n')


1977.2733061612314
1710.4185793622946
1288.3873723470788
1099.1480132801491
903.6936800562227
741.4538711350989
605.001122659824
528.9393984546095
395.1072109291937
345.77646120090435
271.4200441533309
216.11524887500923
201.24945151300895
150.7270130620796
121.4364666279503
107.42766557811362
87.65956852146452
69.9404092961646
53.51635485562993
43.148232573779545
36.01983100153369
28.901169495232978
24.65701114454576
20.679405860195534
16.936169098090634
13.66466581950324
11.27120010322844
9.262971317090653
7.438121429334432
6.3082289055910215
4.841573737697555
4.397997551778146
3.4295085917323047
3.1326874859099103
2.4261300380602724
1.770960085570563
1.6055020961967106
1.355123967167543
0.9840130111573284
0.886940057827982
0.691355419113849
0.6120882682780324
0.47394482256921355
0.40351908194109476
0.32835436342518826
0.29606317769677365
0.22890237894792187
0.19004186786675753
0.17048014113485804
0.1311799750556162
0.12835403427069106
0.10806101062948913
0.08446707260101619
0.068345

66.68483772760368
60.2503755668228
48.91786983332927
38.725075916408
28.738853581498564
27.340622946384165
21.656662394905165
19.11900415819291
14.807504840968127
13.00533496464939
10.832926575781846
7.598867550966291
6.877325024256105
5.9342683148343465
4.4922364748519
3.8289490064215403
3.283374717902527
2.5973221275229124
2.1762071632033044
1.6811770333410492
1.470682883512652
1.2976182222228752
1.0858892539492069
0.8999976237363952
0.7581258065827252
0.6156406623972895
0.5457954655170122
0.4450694692798333
0.36604965976018805
0.2909847561980066
0.2838451595663845
0.2595776833711953
0.24422506974167546
0.19641125071956803
0.1848889865012057
0.1580500254501883
0.15695159539799228
0.13051222975238613
0.11084401975430809
0.10430126975905667
0.0863963936220372
0.07234798428166533
0.06811975557570499
0.05898679209224424
0.06456721824624058
0.055270877359147014
0.05235321727809212
0.04469718569274937
0.04122862176048151
0.030777925292432697
0.031071988948692265
0.03505506238887335
0.02931

In [122]:
df = pd.DataFrame(np.vstack([RMSE_train, RMSE_test]), 
                  index=['rmse_train','rmse_test'])

df = pd.concat([df, df.mean(axis=1).rename('mean'),
                df.std(axis=1).rename('std')], axis=1)

df

,0,1,2,3,4,mean,std
rmse_train,0.000117,0.000019,0.000086,0.000059,0.000130,0.000082,0.000045
rmse_test,0.000116,0.000019,0.000085,0.000062,0.000127,0.000082,0.000044


<h1>Фигачим на данных1</h1>

In [124]:
X = sm
y = df_targ

folds_index = 5
fold_size = round(X.shape[0] / folds_index)
RMSE_test = []
RMSE_train = []

for i in range(folds_index):
    test = X[i * fold_size:(i + 1) * fold_size]
    testT = y[i * fold_size:(i + 1) * fold_size]
    if i == 0:
        train = X[(i + 1) * fold_size:, :]
        trainT = y[(i + 1) * fold_size:]
    else:
        train = X[:i * fold_size, :]
        trainT = y[:i * fold_size]
        if i != 4:
            train = np.concatenate((train, X[(i + 1) * fold_size:, :]))
            trainT = np.concatenate([trainT, y[(i + 1) * fold_size:]])
    
    Target = trainT
    Features = train
    w0, w1, V = sgdFFM(Features, Target, 1e4, 1e-3)

    train_pred = predict(Features, V, w0, w1)

    RMSE_train.append(RMSE(train_pred, Target))
    TargetT = testT
    FeaturesT = test
    test_pred = predict(FeaturesT, V, w0, w1)
    RMSE_test.append(RMSE(test_pred, TargetT))

print('Fold {}'.format(i + 1), '\nОбучающая: \n', train_pred - Target, '\nТестовая: \n', test_pred - TargetT, '\n', '\n')


TypeError: matrix is not square

In [ ]:
df = pd.DataFrame(np.vstack([RMSE_train, RMSE_test]), 
                  index=['rmse_train','rmse_test'])

df = pd.concat([df, df.mean(axis=1).rename('mean'),
                df.std(axis=1).rename('std')], axis=1)

df